In [49]:
import pandas as pd
import requests

In [50]:

lat = 59.9333
lon = 10.7166

url = f"https://api.met.no/weatherapi/nowcast/2.0/complete?lat={lat}&lon={lon}"
headers = {"User-Agent": "WeatherTest/1.0 skybert@gmail.com"}

response = requests.get(url, headers=headers)
result = {}

if response.status_code == 200:
    result = response.json()
else:
    print(f"Error: {response.status_code}")
    print(response.text)

In [51]:
import pytz
from datetime import datetime

def convert_time_stamp(timestamp_str):
    # Deinerer tidssonen for pytz
    utc_timezone = pytz.timezone('UTC')

    # Setter tidssonen vi vil ha det i
    local_timezone = pytz.timezone('Europe/Copenhagen')  # Replace 'Your/Timezone' with your timezone

    # Konverterer stringen til et datetime objekt
    timestamp_utc = datetime.strptime(timestamp_str, '%Y-%m-%dT%H:%M:%SZ')

    # Lokaliserer strengen til utc
    timestamp_utc = utc_timezone.localize(timestamp_utc)

    # Konverterer til vår lokale tidssone som vi satte før
    timestamp_local = timestamp_utc.astimezone(local_timezone)

    # Formatterer strengen slik at den er mer lesbar og gir mening for oss
    local_timestamp_str = timestamp_local.strftime('%Y-%m-%d %H:%M:%S %Z')

    return local_timestamp_str

In [52]:
print(result)

{'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [10.7166, 59.9333, 55]}, 'properties': {'meta': {'updated_at': '2024-02-12T16:09:13Z', 'units': {'air_temperature': 'celsius', 'precipitation_amount': 'mm', 'precipitation_rate': 'mm/h', 'relative_humidity': '%', 'wind_from_direction': 'degrees', 'wind_speed': 'm/s', 'wind_speed_of_gust': 'm/s'}, 'radar_coverage': 'ok'}, 'timeseries': [{'time': '2024-02-12T16:10:00Z', 'data': {'instant': {'details': {'air_temperature': -3.7, 'precipitation_rate': 0.5, 'relative_humidity': 81.0, 'wind_from_direction': 37.1, 'wind_speed': 4.1, 'wind_speed_of_gust': 8.6}}, 'next_1_hours': {'summary': {'symbol_code': 'snow'}, 'details': {'precipitation_amount': 0.4}}}}, {'time': '2024-02-12T16:15:00Z', 'data': {'instant': {'details': {'precipitation_rate': 0.4}}}}, {'time': '2024-02-12T16:20:00Z', 'data': {'instant': {'details': {'precipitation_rate': 0.5}}}}, {'time': '2024-02-12T16:25:00Z', 'data': {'instant': {'details': {'precipitation_ra

In [58]:
# Lage variabler med dictonaries med relevant informasjon

updated_at = result["properties"]["meta"]["updated_at"]
units_dict = result["properties"]["meta"]["units"]


TypeError: list indices must be integers or slices, not str

In [60]:
units_measured_list = result["properties"]["timeseries"]

[{'time': '2024-02-12T16:10:00Z',
  'data': {'instant': {'details': {'air_temperature': -3.7,
     'precipitation_rate': 0.5,
     'relative_humidity': 81.0,
     'wind_from_direction': 37.1,
     'wind_speed': 4.1,
     'wind_speed_of_gust': 8.6}},
   'next_1_hours': {'summary': {'symbol_code': 'snow'},
    'details': {'precipitation_amount': 0.4}}}},
 {'time': '2024-02-12T16:15:00Z',
  'data': {'instant': {'details': {'precipitation_rate': 0.4}}}},
 {'time': '2024-02-12T16:20:00Z',
  'data': {'instant': {'details': {'precipitation_rate': 0.5}}}},
 {'time': '2024-02-12T16:25:00Z',
  'data': {'instant': {'details': {'precipitation_rate': 0.4}}}},
 {'time': '2024-02-12T16:30:00Z',
  'data': {'instant': {'details': {'precipitation_rate': 0.4}}}},
 {'time': '2024-02-12T16:35:00Z',
  'data': {'instant': {'details': {'precipitation_rate': 0.4}}}},
 {'time': '2024-02-12T16:40:00Z',
  'data': {'instant': {'details': {'precipitation_rate': 0.4}}}},
 {'time': '2024-02-12T16:45:00Z',
  'data': {

In [62]:
# Lage noe her som tar tiden og konverterer i formatet jeg har satt nedenfor også joiner det sammen

for i in units_measured_list:
    print(i["time"])

2024-02-12T16:10:00Z
2024-02-12T16:15:00Z
2024-02-12T16:20:00Z
2024-02-12T16:25:00Z
2024-02-12T16:30:00Z
2024-02-12T16:35:00Z
2024-02-12T16:40:00Z
2024-02-12T16:45:00Z
2024-02-12T16:50:00Z
2024-02-12T16:55:00Z
2024-02-12T17:00:00Z
2024-02-12T17:05:00Z
2024-02-12T17:10:00Z
2024-02-12T17:15:00Z
2024-02-12T17:20:00Z
2024-02-12T17:25:00Z
2024-02-12T17:30:00Z
2024-02-12T17:35:00Z
2024-02-12T17:40:00Z
2024-02-12T17:45:00Z
2024-02-12T17:50:00Z
2024-02-12T17:55:00Z
2024-02-12T18:00:00Z


In [54]:
units_dict

{'air_temperature': 'celsius',
 'precipitation_amount': 'mm',
 'precipitation_rate': 'mm/h',
 'relative_humidity': '%',
 'wind_from_direction': 'degrees',
 'wind_speed': 'm/s',
 'wind_speed_of_gust': 'm/s'}

In [55]:
unit_df = pd.DataFrame.from_dict(units_dict, orient='index', columns=['Unit'])
unit_df.index.name = 'Attribute'

In [56]:
unit_df

,Unit
Attribute,
air_temperature,celsius
precipitation_amount,mm
precipitation_rate,mm/h
relative_humidity,%
wind_from_direction,degrees
wind_speed,m/s
wind_speed_of_gust,m/s


In [57]:

converted_time = convert_time_stamp(updated_at)
print(f"Sist oppdatert at {converted_time}")

Sist oppdatert at 2024-02-12 17:09:13 CET
